In [1]:
import pandas as pd
import eclabfiles as ecf
from matplotlib import pyplot as plt
import numpy as np
import glob
from datetime import datetime
from pickle_utils import read_ecf, read_emsi, optimise_emsi_start
%matplotlib qt

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
DATE = '2024-03-19'

In [4]:
folder = 'T:/Team/Szewczyk/Data/' + DATE
ecfile = glob.glob(folder+'/*.mpr')[-1]
datfiles = glob.glob(folder+'/*.dat')

In [5]:
ecfdf, ecf_start = read_ecf(ecfile)

In [12]:
joined_emsi = pd.DataFrame([])
for filename in datfiles:
    emsidf, emsi_start, emsi_end = read_emsi(filename)
    if emsi_end < ecf_start:
        continue
    emsi_shift = optimise_emsi_start(ecfdf, ecf_start, emsidf, emsi_start)
    emsidf.loc[:, 't'] = emsidf.t + emsi_start - ecf_start - emsi_shift
    emsidf.loc[:, 'I'] = emsidf.I * 1000
    joined_emsi = pd.concat([joined_emsi, emsidf], ignore_index=True)
plt.plot(joined_emsi.t, joined_emsi.I)
plt.plot(ecfdf.t, ecfdf.I)

[-6.64592906]
[-6.26203061]
[-6.19859897]
[-6.23791774]
[-6.10738705]
[-5.9434501]
[-6.400361]
[-5.56458697]


In [13]:
ecfdf['emsi'] = np.interp(ecfdf.t, joined_emsi.t, joined_emsi.emsi)
ecfdf['light'] = np.interp(ecfdf.t, joined_emsi.t, joined_emsi.light)

In [14]:
ecfdf.to_pickle(folder+'/data.pkl')